Upload files to S3 bucket for storage.

In [2]:
import pandas as pd

dates1 = pd.date_range(start='2015-12-01', end='2015-12-02', freq='H', closed='left')
dates2 = pd.date_range(start='2016-04-15', end='2016-04-16', freq='H', closed='left')
dates3 = pd.date_range(start='2016-10-30', end='2016-10-31', freq='H', closed='left')

dates = dates1.union(dates2).union(dates3)

datess = []
for i in range(len(dates)):
    datess.append(str(dates[i]).replace(" ", "_") + "_UTC")
dates = datess

In [3]:
import boto3
import pandas as pd
from sagemaker import get_execution_role
import tqdm

role = get_execution_role()

for date in tqdm.tqdm(dates):
    obj = "tweets_spanish_" + date + ".json"
    obj_loc = "../../data/" + obj
    boto3.Session().resource('s3').Bucket("mt5599").Object("tweets/" + obj).upload_file(obj_loc)

100%|██████████| 72/72 [00:45<00:00,  1.59it/s]


Reading in raw dataframes so they can be combined.

In [6]:
import boto3
import pandas as pd
from sagemaker import get_execution_role
import tqdm

role = get_execution_role()

files = []
DataFrameName = []
for date in tqdm.tqdm(dates):
    files.append("s3://mt5599/tweets/tweets_spanish_" + str(date) + ".json")
    DataFrameName.append("tweets_spanish_" + str(date))

dfs = {}

for name, file in tqdm.tqdm(zip(DataFrameName, files)):
    dfs[name] = pd.read_json(file)

100%|██████████| 72/72 [00:00<00:00, 329683.28it/s]
72it [02:28,  2.06s/it]


Combining handles dataset and uploading it to S3 bucket.

In [7]:
df = pd.concat(dfs.values(), ignore_index=True)
df

id                  DateTime  \
0        671493915590057984 2015-12-01 00:59:59+00:00   
1        671493915497885696 2015-12-01 00:59:59+00:00   
2        671493915342688256 2015-12-01 00:59:59+00:00   
3        671493914801643520 2015-12-01 00:59:59+00:00   
4        671493914176724992 2015-12-01 00:59:59+00:00   
...                     ...                       ...   
1655961  792863655096442880 2016-10-30 23:00:00+00:00   
1655962  792863654274367488 2016-10-30 23:00:00+00:00   
1655963  792863654089723904 2016-10-30 23:00:00+00:00   
1655964  792863653401722880 2016-10-30 23:00:00+00:00   
1655965  792863653129285632 2016-10-30 23:00:00+00:00   

                                             tweet_content cashtags  \
0                           mi fin de semana estuvo cabron     None   
1        Ya me descarge el app😁 ahora mati no me va a p...     None   
2               @gabyespinony Dejame buscar y te aviso !!!     None   
3                               Sin dudas la mejor gestión     None   
4                             Elbo Lita jajajaja que cldo!     None   
...                                                    ...      ...   
1655961  @CFKArgentina con las propiedades  de Báez,cua...     None   
1655962  Temperatura: 22,4°C S.Térmica 22,4°C PR 12,7°C...     None   
1655963  que ondon Ramón con el camarón, que hongo joro...     None   
1655964  Extrañaba esto ☀ @ Nacimiento. Región Bio Bio ...     None   
1655965            Bárbara y Adara a la calle ya #DBT8GH17     None   

                                               coordinates      hashtags lang  \
0        Coordinates(longitude=-67.054105, latitude=18....          None   es   
1        Coordinates(longitude=-58.346384, latitude=-34...          None   es   
2        Coordinates(longitude=-73.3540725, latitude=0....          None   es   
3        Coordinates(longitude=-64.3195, latitude=-31.5...          None   es   
4        Coordinates(longitude=-64.389839, latitude=-33...          None   es   
...                                                    ...           ...  ...   
1655961  Coordinates(longitude=-63.2919697, latitude=-3...          None   es   
1655962  Coordinates(longitude=-58.54916667, latitude=-...          None   es   
1655963  Coordinates(longitude=-97.43391558, latitude=3...          None   es   
1655964  Coordinates(longitude=-72.6704209, latitude=-3...          None   es   
1655965  Coordinates(longitude=-8.0037342, latitude=42....  ['DBT8GH17']   es   

         like_count media                                    mentioned_users  \
0                 1  None                                               None   
1                 1  None                                               None   
2                 0  None  [User(username='gabyespinony', id=2769738383, ...   
3                 0  None                                               None   
4                 0  None                                               None   
...             ...   ...                                                ...   
1655961           0  None  [User(username='CFKArgentina', id=138814032, d...   
1655962           1  None                                               None   
1655963           0  None                                               None   
1655964           0  None                                               None   
1655965           4  None                                               None   

         ...                             user_profile_image_url  \
0        ...  https://pbs.twimg.com/profile_images/160177785...   
1        ...  https://pbs.twimg.com/profile_images/153241012...   
2        ...  https://pbs.twimg.com/profile_images/152098656...   
3        ...  https://pbs.twimg.com/profile_images/160430215...   
4        ...  https://pbs.twimg.com/profile_images/140897405...   
...      ...                                                ...   
1655961  ...  https://pbs.twimg.com/profile_images/149981312...   
1655962  ...  htt

Extracting all handles from Argentina.

In [10]:
df.to_json("../../data/handles_combined.json")

In [2]:
import pandas as pd

df_a2_1 = pd.read_json("s3://mt5599/tweets/handles_combined.json")
df_a2_1

id            DateTime  \
0        671493915590057984 2015-12-01 00:59:59   
1        671493915497885696 2015-12-01 00:59:59   
2        671493915342688256 2015-12-01 00:59:59   
3        671493914801643520 2015-12-01 00:59:59   
4        671493914176724992 2015-12-01 00:59:59   
...                     ...                 ...   
1655961  792863655096442880 2016-10-30 23:00:00   
1655962  792863654274367488 2016-10-30 23:00:00   
1655963  792863654089723904 2016-10-30 23:00:00   
1655964  792863653401722880 2016-10-30 23:00:00   
1655965  792863653129285632 2016-10-30 23:00:00   

                                             tweet_content cashtags  \
0                           mi fin de semana estuvo cabron     None   
1        Ya me descarge el app😁 ahora mati no me va a p...     None   
2               @gabyespinony Dejame buscar y te aviso !!!     None   
3                               Sin dudas la mejor gestión     None   
4                             Elbo Lita jajajaja que cldo!     None   
...                                                    ...      ...   
1655961  @CFKArgentina con las propiedades  de Báez,cua...     None   
1655962  Temperatura: 22,4°C S.Térmica 22,4°C PR 12,7°C...     None   
1655963  que ondon Ramón con el camarón, que hongo joro...     None   
1655964  Extrañaba esto ☀ @ Nacimiento. Región Bio Bio ...     None   
1655965            Bárbara y Adara a la calle ya #DBT8GH17     None   

                                               coordinates      hashtags lang  \
0        Coordinates(longitude=-67.054105, latitude=18....          None   es   
1        Coordinates(longitude=-58.346384, latitude=-34...          None   es   
2        Coordinates(longitude=-73.3540725, latitude=0....          None   es   
3        Coordinates(longitude=-64.3195, latitude=-31.5...          None   es   
4        Coordinates(longitude=-64.389839, latitude=-33...          None   es   
...                                                    ...           ...  ...   
1655961  Coordinates(longitude=-63.2919697, latitude=-3...          None   es   
1655962  Coordinates(longitude=-58.54916667, latitude=-...          None   es   
1655963  Coordinates(longitude=-97.43391558, latitude=3...          None   es   
1655964  Coordinates(longitude=-72.6704209, latitude=-3...          None   es   
1655965  Coordinates(longitude=-8.0037342, latitude=42....  ['DBT8GH17']   es   

         like_count media                                    mentioned_users  \
0                 1  None                                               None   
1                 1  None                                               None   
2                 0  None  [User(username='gabyespinony', id=2769738383, ...   
3                 0  None                                               None   
4                 0  None                                               None   
...             ...   ...                                                ...   
1655961           0  None  [User(username='CFKArgentina', id=138814032, d...   
1655962           1  None                                               None   
1655963           0  None                                               None   
1655964           0  None                                               None   
1655965           4  None                                               None   

         ...                             user_profile_image_url  \
0        ...  https://pbs.twimg.com/profile_images/160177785...   
1        ...  https://pbs.twimg.com/profile_images/153241012...   
2        ...  https://pbs.twimg.com/profile_images/152098656...   
3        ...  https://pbs.twimg.com/profile_images/160430215...   
4        ...  https://pbs.twimg.com/profile_images/140897405...   
...      ...                                                ...   
1655961  ...  https://pbs.twimg.com/profile_images/149981312...   
1655962  ...  https://pbs.twimg.com/profile_images/160486963...   
1655963  ...  https://

In [5]:
# number of tweets collected
df_a2_1 = df_a1_1.drop_duplicates("id")
df_a2_1.shape[0]

1655473

In [6]:
# number of user handles
len(df_a2_1.username.unique())

417185

In [9]:
# number of tweets geotagged with Argentina
ar = df_a2_1[df_a1_1.place_country_code == "AR"]
ar.shape[0]

629803

In [10]:
# number of user handles from argentina
len(ar.username.unique())

113613

In [16]:
import pandas as pd

df_a1_2 = pd.read_feather("s3://mt5599/raw_tweets/2016/df_2016_tweets_cleaned_with_place.feather")
df_a1_2

id            DateTime  \
0        743561801745338368 2016-06-16 21:52:02   
1        743064119754919936 2016-06-15 12:54:26   
2        742934385624809472 2016-06-15 04:18:55   
3        726430125286264832 2016-04-30 15:16:52   
4        684363985974464512 2016-01-05 13:21:03   
...                     ...                 ...   
7686538  649582930189156352 2015-10-01 13:53:33   
7686539  649561708587954176 2015-10-01 12:29:14   
7686540  649446900601483264 2015-10-01 04:53:01   
7686541  649435258853527552 2015-10-01 04:06:46   
7686542  649434917596631040 2015-10-01 04:05:24   

                                             tweet_content cashtags  \
0                                😁 https://t.co/eq2i7lxQkD     None   
1                                           Morning ☕️🌲🍂🌾🍃     None   
2                            👬👭❤️🤗 https://t.co/nMQNrePMXZ     None   
3                 Se van los 25 :/ https://t.co/RddGa6gT4T     None   
4                                           Godo Morning 👓     None   
...                                                    ...      ...   
7686538  El bus va vacío pero no me voy a sentar  com t...     None   
7686539              Buenos días ! \nbienvenido octubre 👏🏼     None   
7686540           Y terminar comiendo una Queso de Mc 😊 😋🍔     None   
7686541     Gracias Mae por su culpa estoy que me vomito 😤     None   
7686542  Un Mae se acaba de echar un una clase de perfu...     None   

                                               coordinates hashtags lang  \
0        Coordinates(longitude=-85.3044586, latitude=8....     None  und   
1        Coordinates(longitude=-85.3044586, latitude=8....     None   en   
2                                                     None     None  und   
3                                                     None     None   es   
4        Coordinates(longitude=-84.851172, latitude=9.9...     None   en   
...                                                    ...      ...  ...   
7686538  Coordinates(longitude=-84.2022247, latitude=9....     None   es   
7686539  Coordinates(longitude=-85.4422302, latitude=9....     None   es   
7686540                                               None     None   es   
7686541  Coordinates(longitude=-84.1299362, latitude=9....     None   es   
7686542  Coordinates(longitude=-84.1299362, latitude=9....     None   es   

         like_count                                              media  \
0                 1  [Photo(previewUrl='https://pbs.twimg.com/media...   
1                 0                                               None   
2                 0  [Photo(previewUrl='https://pbs.twimg.com/media...   
3                 3  [Photo(previewUrl='https://pbs.twimg.com/media...   
4                 1                                               None   
...             ...                                                ...   
7686538           0                                               None   
7686539           0                                               None   
7686540           0                                               None   
7686541           0                                               None   
7686542           0                                               None   

        mentioned_users  ...  \
0                  None  ...   
1                  None  ...   
2                  None  ...   
3                  None  ...   
4                  None  ...   
...                 ...  ...   
7686538            None  ...   
7686539            None  ...   
7686540            None  ...   
7686541            None  ...   
7686542            None  ...   

                                    user_profile_image_url  \
0        https://pbs.twimg.com/profile_images/154994465...   
1        https://pbs.twimg.com/profile_images/154994465...   
2        https://pbs.twimg.com/profile_images/154994465...   
3        https://pbs.twimg.com/profile_images/154994465...   
4        https://pbs.twimg.com/profile_images/154994465...   


In [17]:
len(df_a2_2.username.unique())

5604

In [32]:
df_a2_2[(df_a2_2.place_country_code != "None") & (df_a2_2.place_country_code != None)]

id            DateTime  \
0        743561801745338368 2016-06-16 21:52:02   
1        743064119754919936 2016-06-15 12:54:26   
4        684363985974464512 2016-01-05 13:21:03   
5        660450319030087680 2015-10-31 13:36:41   
7        658366416039493632 2015-10-25 19:36:00   
...                     ...                 ...   
7686537  649643897006592000 2015-10-01 17:55:49   
7686538  649582930189156352 2015-10-01 13:53:33   
7686539  649561708587954176 2015-10-01 12:29:14   
7686541  649435258853527552 2015-10-01 04:06:46   
7686542  649434917596631040 2015-10-01 04:05:24   

                                             tweet_content cashtags  \
0                                😁 https://t.co/eq2i7lxQkD     None   
1                                           Morning ☕️🌲🍂🌾🍃     None   
4                                           Godo Morning 👓     None   
5                             ☀️🌴🌊 https://t.co/RpNSGNdTxP     None   
7                                    Quiero chicharrones 🐖     None   
...                                                    ...      ...   
7686537  No le sentí, pero en este edificio que miedo😬 ...     None   
7686538  El bus va vacío pero no me voy a sentar  com t...     None   
7686539              Buenos días ! \nbienvenido octubre 👏🏼     None   
7686541     Gracias Mae por su culpa estoy que me vomito 😤     None   
7686542  Un Mae se acaba de echar un una clase de perfu...     None   

                                               coordinates  \
0        Coordinates(longitude=-85.3044586, latitude=8....   
1        Coordinates(longitude=-85.3044586, latitude=8....   
4        Coordinates(longitude=-84.851172, latitude=9.9...   
5        Coordinates(longitude=-84.851172, latitude=9.9...   
7        Coordinates(longitude=-84.851172, latitude=9.9...   
...                                                    ...   
7686537  Coordinates(longitude=-84.1004715, latitude=9....   
7686538  Coordinates(longitude=-84.2022247, latitude=9....   
7686539  Coordinates(longitude=-85.4422302, latitude=9....   
7686541  Coordinates(longitude=-84.1299362, latitude=9....   
7686542  Coordinates(longitude=-84.1299362, latitude=9....   

                                          hashtags lang  like_count  \
0                                             None  und           1   
1                                             None   en           0   
4                                             None   en           1   
5                                             None  und           2   
7                                             None   es           1   
...                                            ...  ...         ...   
7686537  ['TemblorCR', 'Asamblea', 'EdificioSion']   es           0   
7686538                                       None   es           0   
7686539                                       None   es           0   
7686541                                       None   es           0   
7686542                                       None   es           0   

                                                     media mentioned_users  \
0        [Photo(previewUrl='https://pbs.twimg.com/media...            None   
1                                                     None            None   
4                                                     None            None   
5        [Photo(previewUrl='https://pbs.twimg.com/media...            None   
7                                                     None            None   
...                                                    ...             ...   
7686537                                               None            None   
7686538                                               None            None   
7686539                                               None            None   
7686541                                               None            None   
7686542                                               None            None   

         ...            

In [39]:
# Investigating countries by tweet number (geotags vs no geotags)

loc_vc = df_a2_2.place_country.value_counts(dropna=True).rename_axis('unique_values').reset_index(name='counts')
loc_vc = loc_vc[loc_vc.unique_values != "None"].reset_index(drop=True)
loc_total = sum(loc_vc.counts)
#only_loc_vc = mdf_loc_only.country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#only_loc_total = sum(only_loc_vc.counts)

vc = pd.DataFrame(data = {"loc": [],
                          "loc_counts": []})
          #                "loc_prop": []})
      #                   "only_loc": [],
      #                   "only_loc_counts": [],
      #                   "only_loc_prop": []})

n = 20
for i in range(n):
    vc.loc[i, "loc"] = loc_vc.loc[i, "unique_values"]
    vc.loc[i, "loc_counts"] = loc_vc.loc[i, "counts"]
   # vc.loc[i, "loc_prop"] = loc_vc.loc[i, "counts"] / loc_total
    
 #   vc.loc[i, "only_loc"] = only_loc_vc.loc[i, "unique_values"]
 #   vc.loc[i, "only_loc_counts"] = only_loc_vc.loc[i, "counts"]
 #   vc.loc[i, "only_loc_prop"] = only_loc_vc.loc[i, "counts"] / only_loc_total

vc.loc[n, "loc"] = "Other"
vc.loc[n, "loc_counts"] = loc_total - sum(loc_vc.loc[0:(n-1), "counts"])
#vc.loc[n+1, "loc_prop"] = vc.loc[n+1, "loc_counts"] / loc_total
#vc.loc[n+1, "only_loc"] = "Other"
#vc.loc[n+1, "only_loc_counts"] = only_loc_total - sum(only_loc_vc.loc[0:n, "counts"])
#vc.loc[n+1, "only_loc_prop"] = vc.loc[n+1, "only_loc_counts"] / only_loc_total

#vc.reset_index(inplace=True, drop=True)
#vc.drop(0, axis=0, inplace=True)
vc

loc  loc_counts
0       Costa Rica   1599239.0
1    United States    109470.0
2        Nicaragua     47431.0
3        Argentina     27279.0
4            Spain     22944.0
5           Mexico     20541.0
6   United Kingdom     10514.0
7           Canada      9717.0
8           Panama      9545.0
9         Colombia      9155.0
10          Brazil      8879.0
11          Russia      8464.0
12           Chile      6613.0
13          Poland      5750.0
14       Venezuela      5712.0
15       Australia      4057.0
16           India      3993.0
17           Italy      3914.0
18         Germany      3783.0
19        Malaysia      3763.0
20           Other     72970.0

In [40]:
vc.loc_counts.sum()

1993733.0

In [29]:
loc_vc.counts.sum()

1993733

In [36]:
loc_vc.loc[0:n, "counts"]

0     1599239
1      109470
2       47431
3       27279
4       22944
5       20541
6       10514
7        9717
8        9545
9        9155
10       8879
11       8464
12       6613
13       5750
14       5712
15       4057
16       3993
17       3914
18       3783
19       3763
20       3717
Name: counts, dtype: int64

In [37]:
loc_vc.counts

1993733

In [13]:
import pandas as pd

df_ = pd.read_feather("s3://mt5599/tweets/spanish_tweets_2016.feather")
df_2016

index                  id                  DateTime  \
0                0  676710697422725120 2015-12-15 10:29:37+00:00   
1                1  674648800020033536 2015-12-09 17:56:22+00:00   
2                2  671493815635746816 2015-12-01 00:59:36+00:00   
3                3  669967909456879616 2015-11-26 19:56:11+00:00   
4                4  668096786893021184 2015-11-21 16:01:01+00:00   
...            ...                 ...                       ...   
56458598  56458598  652645146643857408 2015-10-10 00:41:43+00:00   
56458599  56458599  652642515749834752 2015-10-10 00:31:15+00:00   
56458600  56458600  652642476738654208 2015-10-10 00:31:06+00:00   
56458601  56458601  652641780056346624 2015-10-10 00:28:20+00:00   
56458602  56458602  652641082996617216 2015-10-10 00:25:34+00:00   

                                                coordinates  \
0         Coordinates(longitude=-57.5592828, latitude=-3...   
1         Coordinates(longitude=-57.5548276, latitude=-3...   
2         Coordinates(longitude=-57.5548158, latitude=-3...   
3         Coordinates(longitude=-57.5591638, latitude=-3...   
4         Coordinates(longitude=-57.5548388, latitude=-3...   
...                                                     ...   
56458598  Coordinates(longitude=-58.447767, latitude=-34...   
56458599  Coordinates(longitude=-58.447767, latitude=-34...   
56458600  Coordinates(longitude=-58.447767, latitude=-34...   
56458601  Coordinates(longitude=-58.447767, latitude=-34...   
56458602  Coordinates(longitude=-58.447767, latitude=-34...   

                                                      place        username  \
0         Place(fullName='Mar del Plata, Argentina', nam...     darcena2553   
1         Place(fullName='Mar del Plata, Argentina', nam...     darcena2553   
2         Place(fullName='Mar del Plata, Argentina', nam...     darcena2553   
3         Place(fullName='Mar del Plata, Argentina', nam...     darcena2553   
4         Place(fullName='Mar del Plata, Argentina', nam...     darcena2553   
...                                                     ...             ...   
56458598  Place(fullName='Almirante Brown, Argentina', n...  daniimercado89   
56458599  Place(fullName='Almirante Brown, Argentina', n...  daniimercado89   
56458600  Place(fullName='Almirante Brown, Argentina', n...  daniimercado89   
56458601  Place(fullName='Almirante Brown, Argentina', n...  daniimercado89   
56458602  Place(fullName='Almirante Brown, Argentina', n...  daniimercado89   

                       user_location             place_full_name  \
0         No soy unica soy diferente    Mar del Plata, Argentina   
1         No soy unica soy diferente    Mar del Plata, Argentina   
2         No soy unica soy diferente    Mar del Plata, Argentina   
3         No soy unica soy diferente    Mar del Plata, Argentina   
4         No soy unica soy diferente    Mar del Plata, Argentina   
...                              ...                         ...   
56458598                              Almirante Brown, Argentina   
56458599                              Almirante Brown, Argentina   
56458600                              Almirante Brown, Argentina   
56458601                              Almirante Brown, Argentina   
56458602                              Almirante Brown, Argentina   

               place_name place_type place_country place_country_code  \
0           Mar del Plata       city     Argentina                 AR   
1           Mar del Plata       city     Argentina                 AR   
2           Mar del Plata       city     Argentina                 AR   
3           Mar del Plata       city     Argentina                 AR   
4           Mar del Plata       city     Argentina                 AR   
...                   ...        ...           ...                ...   
56458598  Almirante Brown       city     Argentina                 AR   
56458599  Almirante Brown       city     Argentina                 AR   
56458600  Almira

In [15]:
len(df_2016.username.unique())

89329

In [14]:
# Investigating countries by tweet number (geotags vs no geotags)

loc_vc = df_2016.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
loc_total = sum(loc_vc.counts)
#only_loc_vc = mdf_loc_only.country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#only_loc_total = sum(only_loc_vc.counts)

vc = pd.DataFrame(data = {"loc": [],
                          "loc_counts": [],
                          "loc_prop": []})
      #                   "only_loc": [],
      #                   "only_loc_counts": [],
      #                   "only_loc_prop": []})

n = 20
for i in range(n):
    vc.loc[i, "loc"] = loc_vc.loc[i, "unique_values"]
    vc.loc[i, "loc_counts"] = loc_vc.loc[i, "counts"]
    vc.loc[i, "loc_prop"] = loc_vc.loc[i, "counts"] / loc_total
    
 #   vc.loc[i, "only_loc"] = only_loc_vc.loc[i, "unique_values"]
 #   vc.loc[i, "only_loc_counts"] = only_loc_vc.loc[i, "counts"]
 #   vc.loc[i, "only_loc_prop"] = only_loc_vc.loc[i, "counts"] / only_loc_total

vc.loc[n+1, "loc"] = "Other"
vc.loc[n+1, "loc_counts"] = loc_total - sum(loc_vc.loc[0:n, "counts"])
vc.loc[n+1, "loc_prop"] = vc.loc[n+1, "loc_counts"] / loc_total
#vc.loc[n+1, "only_loc"] = "Other"
#vc.loc[n+1, "only_loc_counts"] = only_loc_total - sum(only_loc_vc.loc[0:n, "counts"])
#vc.loc[n+1, "only_loc_prop"] = vc.loc[n+1, "only_loc_counts"] / only_loc_total

vc.reset_index(inplace=True, drop=True)
#vc.drop(0, axis=0, inplace=True)
vc

loc  loc_counts  loc_prop
0            Argentina  55750284.0  0.987526
1        United States    152922.0  0.002709
2               Brazil    128736.0  0.002280
3                Chile     68897.0  0.001220
4              Uruguay     55470.0  0.000983
5                Spain     36817.0  0.000652
6               Mexico     30015.0  0.000532
7             Paraguay     29852.0  0.000529
8             Colombia     18637.0  0.000330
9                Italy     15075.0  0.000267
10      United Kingdom     14845.0  0.000263
11           Venezuela     14511.0  0.000257
12             Ecuador     13180.0  0.000233
13              France     12457.0  0.000221
14              Russia      9023.0  0.000160
15                Peru      8329.0  0.000148
16             Germany      6775.0  0.000120
17               Japan      5729.0  0.000101
18  Dominican Republic      5140.0  0.000091
19              Canada      5104.0  0.000090
20               Other     67663.0  0.001199

In [12]:
! pip install feather-format

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached feather-format-0.4.1.tar.gz (3.2 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2453 sha256=cb8f59798d5ee61ecc7e580075bbd78a411a8bc0a543a366b3679f82d36a20f2
  Stored in directory: /root/.cache/pip/wheels/6f/77/1d/0a36541cf0c2bb1c7bf319185290de60883f3edb8fccbe8a40
Successfully built feather-format


In [14]:
boto3.Session().resource('s3').Bucket("mt5599").Object("tweets/handles_combined.json").upload_file("../../data/handles_combined.json")

In [ ]:
# number of tweets gathered in spanish by country code (all geotagged)

df.place_country_code.value_counts(dropna=False)

In [ ]:
ar_usernames = df[df.place_country_code == "AR"]["username"].unique()
len(ar_usernames)

Saving handles to S3 bucket.

In [ ]:
with open("../../data/spanish_speakers_2016.txt", "w") as f:
    for handle in ar_usernames:
        f.write(f"{handle}\n")

boto3.Session().resource('s3').Bucket("mt5599").Object("tweets/spanish_speakers_2016.txt").upload_file("../../data/spanish_speakers_2016.txt")